In [17]:
import os
import math
import pandas as pd
import re

In [18]:
folder_path    = r"Z:\UriMons\Fiber Photometry\adBNST_eEPA_GCaMP8s\1st_batch\Behavior\Bins_output"
rows_per_bin   = 300             # e.g. 900 rows per bin
bin_label      = "10 Sec Bin"    # whatever you want to call the new column

In [19]:
output_filename = os.path.join(
    folder_path,
    f"Binary_Grooming_Data_{bin_label.replace(' ', '')}.xlsx"
)
skip_filename = os.path.basename(output_filename)

In [20]:
# only match filenames that start with 'M' + one or more digits, then anything, then '.xlsx'
pattern = re.compile(r'^M\d+.*\.xlsx$', re.IGNORECASE)

with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
    for filename in os.listdir(folder_path):
        # skip our own output
        if filename == skip_filename:
            continue
        # enforce M+digits at start, and .xlsx at end
        if not pattern.match(filename):
            continue

        df = pd.read_excel(os.path.join(folder_path, filename), engine='openpyxl')
        n = len(df)
        n_bins = math.ceil(n / rows_per_bin)

        # sum grooming per block
        bin_sums = []
        for i in range(n_bins):
            start = i * rows_per_bin
            end   = min((i + 1) * rows_per_bin, n)
            bin_sums.append(df['grooming'].iloc[start:end].sum())

        # attach new column
        new_col = [None] * n
        for i, s in enumerate(bin_sums):
            new_col[i] = s
        df[bin_label] = new_col

        # write out
        sheet_name = os.path.splitext(filename)[0]
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Done! Output saved to:\n  {output_filename}")

Done! Output saved to:
  Z:\UriMons\Fiber Photometry\adBNST_eEPA_GCaMP8s\1st_batch\Behavior\Bins_output\Binary_Grooming_Data_10SecBin.xlsx
